In [10]:
import pandas as pd

In [467]:
tariff_broadband_df = pd.read_excel(r'Тариф.xlsx', names=['account','connected', 'tariff', 'IP', 'lease'])
payment_broadband_df = pd.read_excel(r'Оплата.xlsx', names=['account', 'datetime', 'amount', 'method']);
CC_call_df = pd.read_excel(r'Обращения в КЦ.xlsx', names=['date', 'time', 'duration', 'account', 'region', 'theme', 'type'])
change_tariff_broadband_df = pd.read_excel(r'Изменения тарифов.xlsx', names=['account', 'datetime', 'new_tariff', 'old_tariff']);

tariff_broadband_df['connected'] = tariff_broadband_df['connected'].astype('datetime64').dt.date
payment_broadband_df['datetime'] = payment_broadband_df['datetime'].astype('datetime64').dt.date
CC_call_df['date'] = CC_call_df['date'].astype('datetime64').dt.date
change_tariff_broadband_df['datetime'] = change_tariff_broadband_df['datetime'].astype('datetime64').dt.date

In [58]:
tariff_broadband_df.isna().sum()

account      0
connected    0
tariff       0
IP           0
lease        0
dtype: int64

In [255]:
payment_broadband_df.isna().sum()

account     0
datetime    0
amount      0
method      0
dtype: int64

In [256]:
CC_call_df.isna().sum()

date        0
time        0
duration    0
account     0
region      0
theme       0
type        0
dtype: int64

In [257]:
change_tariff_broadband_df.isna().sum()

account           0
datetime          0
new_tariff        0
old_tariff    16751
dtype: int64

<h4>1. Укажите три самых частых тарифа</h4>

In [560]:
tariff_broadband_df.tariff.value_counts() #План А          10167
                                          #Средний план     2397
                                          #Камбэк 250       2396

План А          10167
Средний план     2397
Камбэк 250       2396
Крупный план      766
Камбэк 450        591
Гигаплан          267
Космос            149
Старт              94
Name: tariff, dtype: int64

<h4>2. Укажите месяц с самым большим количеством подключений</h4>

In [155]:
tariff_broadband_df_time = pd.read_excel(r'Тариф.xlsx', index_col = 'connected')

In [558]:
tariff_broadband_df_time.account\
                        .resample('1m')\
                        .count()\
                        .sort_values(ascending=False) #самое большое количество подключений было в октябре

connected
2018-10-31    1916
2018-08-31    1808
2018-12-31    1708
2018-09-30    1678
2018-11-30    1614
2018-07-31    1597
2018-06-30    1501
2018-05-31    1157
2018-03-31    1074
2018-04-30    1072
2018-02-28     937
2018-01-31     765
Name: account, dtype: int64

<h4>3. Найдите корреляцию между наличием у абонента статического IP адреса и наличием роутера в аренде. Что можно сказать о связи друг с другом этих свойств?</h4>

In [195]:
print(tariff_broadband_df.IP.sum())           #248 заявок на статический IP
print(tariff_broadband_df.lease.sum())        #3992 заявки на роутер
tariff_broadband_df[['IP', 'lease']].corr()   #переменные никак не связаны

248
3992


,IP,lease
IP,1.000000,-0.040385
lease,-0.040385,1.000000


<h4>4. Определите среднее, моду, медиану, 1 и 3 квартили (Q1 и Q3) поступивших платежей</h4>

In [245]:
payment_broadband_df[payment_broadband_df.amount>0]\
                                        [['amount']]\
                                        .median()

amount    500.0
dtype: float64

In [247]:
payment_broadband_df[payment_broadband_df.amount>0]\
                                        [['amount']]\
                                        .mode()

,amount
0,500.0


In [246]:
payment_broadband_df[payment_broadband_df.amount>0]\
                                        [['amount']]\
                                        .describe()

,amount
count,160631.000000
mean,482.156944
std,392.443483
min,0.120000
25%,250.000000
50%,500.000000
75%,590.000000
max,15000.000000


<h4>5. Определите месяц с наибольшей суммой платежей</h4>

In [250]:
payment_broadband_df_time = pd.read_excel(r'Оплата.xlsx', index_col = 'datetime')

In [259]:
payment_broadband_df_time.amount\
                         .resample('1m')\
                         .sum()\
                         .sort_values(ascending=False) #декабрь месяц с наибольшей суммой платежей

datetime
2018-12-31    11129519.09
2018-11-30     9873420.65
2018-10-31     9034717.60
2018-09-30     8342754.22
2018-08-31     7268916.62
2018-07-31     6418289.29
2018-06-30     5579337.87
2018-05-31     4980803.09
2018-04-30     4410217.18
2018-03-31     4045453.52
2018-02-28     3415200.55
2018-01-31     2890035.06
Name: amount, dtype: float64

<h4>6. Найдите 3 договора, заплативших наибольшую сумму</h4>

In [263]:
payment_broadband_df_time.sort_values(by='amount', ascending=False).head(3)

,account,amount,method
datetime,,,
2018-05-20,100667,15000.0,Сбербанк
2018-03-23,781442,13000.0,OSMP
2018-04-24,184287,11246.7,OSMP


<h4>7. Найдите самый частый метод оплаты у клиентов, подключившихся в октябре</h4>

In [273]:
payment_broadband_df_time.method['2018-10'].value_counts() #Альфа банк самый частый метод оплаты в октябре

Альфа                 7761
yandex                5431
Сбербанк              3238
OSMP                  1175
Inplat                 980
МКБ                    202
Банковский перевод      15
Name: method, dtype: int64

<h4>8. Найдите 3 договора, на которых наиболее часто меняли тарифы</h4>

In [818]:
change_tariff_broadband_df.account.value_counts().head(5)
#договор №105594    26 раз менял тариф за 2018 год
#договор №157175    22 раз менял тариф за 2018 год
#договор №298677    20 раз менял тариф за 2018 год

105594    26
157175    22
298677    20
157790    19
194453    16
Name: account, dtype: int64

<h4>9. Укажите тариф, с которого "уходили" (переключались на другой) чащей всего</h4>

In [817]:
change_tariff_broadband_df.old_tariff.value_counts().head(5)
#не считая пропущенных значений, самый частый тариф, который меняли - План А

План А          1501
Средний план    1081
Камбэк 250       699
Крупный план     493
Камбэк 450       226
Name: old_tariff, dtype: int64

<h4>10. Укажите договор клиента, который подключившись в 2018 году, обращался в КЦ чаще всего</h4>

In [816]:
CC_call_df.account.value_counts().head(5)

270504     218
1220565    179
125703     100
224455      91
2288513     90
Name: account, dtype: int64

In [410]:
cnct_accs = list(tariff_broadband_df.account) #все договоры подключившиеся в 2018 году

In [411]:
sprt_accs = list(CC_call_df.account.value_counts().index) #все договоры обращавшиеся в поддержку 
                                                          #отсортированные по количеству обращений в порядке убывания

In [661]:
%%timeit
top_account, flag = 0, 0
for account in sprt_accs:
    if account in cnct_accs and flag<1:
        top_account = account
        flag+=1
    elif flag==1:
        break

1.69 ms ± 11.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [408]:
top_account #договор клиента, который подключившись в 2018 году, обращался в КЦ чаще всего

244516

In [394]:
sum(CC_call_df.account == top_account) #договор 244516 обращался в КЦ 72 раза

72

<h4>11. Укажите шестичасовой интервал, в котором было больше всего обращений в КЦ</h4>

In [516]:
CC_call_df_time = pd.read_excel(r'Обращения в КЦ.xlsx', 
                                parse_dates={'datetime': ['date', 'time']}, 
                                index_col='datetime')

In [815]:
CC_call_df_time.resample('6h')\
               .count()\
               .sort_values(by='duration', 
                            ascending=False)\
                .head(5)

,duration,account,region,theme,type
datetime,,,,,
2018-06-23 12:00:00,434,434,434,434,434
2018-11-24 12:00:00,395,395,395,395,395
2018-10-07 12:00:00,319,319,319,319,319
2018-11-24 18:00:00,301,301,301,301,301
2018-10-19 12:00:00,291,291,291,291,291


In [557]:
len(CC_call_df_time['2018-06-23 12':'2018-06-23 17']) #check

434

In [ ]:
#Шестичасовой интервал, в котором было больше всего обращений в КЦ: [2018-06-23 12:00:00 -> 2018-06-23 18:00:00)

<h4>12. Определите среднее время разговора с КЦ клиентов из г. Обнинск</h4>

In [572]:
CC_call_df_time[CC_call_df_time.region=='г. Обнинск'].duration.mean()

232.44

<h4>13. Укажите 5 наиболее часто встречающихся типов обращений в КЦ (в порядке убывания их кол-ва) клиентов, которые подключились в 2018 году и сделали платежей на сумму от 10 000 руб.</h4>

In [735]:
cnctd_2018 = list(tariff_broadband_df.account) #все подключившиеся в 2018

In [736]:
pmnt_df = payment_broadband_df[['amount', 'account']]\
                                    .groupby('account')\
                                    .sum()
pmnt_df = pmnt_df[pmnt_df>10000].dropna()

In [737]:
pmnt_10k = pmnt_df.index #все заплатившие более 10000 руб.

In [812]:
%%timeit
cnctd_and_pmnt = [account for account in pmnt_10k if account in cnctd_2018]          
#подключившиеся в 2018 и заплатившие более 10000 руб

10.7 ms ± 47.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [795]:
CC_call_df_subset = CC_call_df[['account', 'type']].set_index('account')

In [820]:
CC_call_df_subset[CC_call_subset.index.isin(cnctd_and_pmnt)].type.value_counts().head(5)
#Проблема с Интернет                             39
#Иная консультация                               29
#Общая проблема                                  12
#Повторное обращение по техническому ТТ          11
#Вопрос по настройке оборудования                11

Проблема с Интернет                       39
Иная консультация                         29
Общая проблема                            12
Повторное обращение по техническому ТТ    11
Вопрос по настройке оборудования          11
Name: type, dtype: int64